# Convolutional Neural Networks in TensorFlow


In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


## Dataset
Here we're importing the MNIST dataset and using a convenient TensorFlow function to batch, scale, and one-hot encode the data.

In [3]:
from examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

#parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# number of samples to calculate validation and accuracy 
test_valid_size = 256 # if you run out of memory decrease this

# network parameters
n_classes = 10 # MNIST total classes
dropout = 0.75 # probability to keep units

Extracting ./train-images-idx3-ubyte.gz
Extracting ./train-labels-idx1-ubyte.gz
Extracting ./t10k-images-idx3-ubyte.gz
Extracting ./t10k-labels-idx1-ubyte.gz


## Weights and Biases
We will create 3 layers alternating between convolutions and max pooling followed by a fully connected and output layer. We begin by defining the necessary weights and biases

In [4]:
weights = {
    "wc1": tf.Variable(tf.random_normal([5,5,1,32])),
    "wc2": tf.Variable(tf.random_normal([5,5,32,64])),
    "wd1": tf.Variable(tf.random_normal([7*7*64,1024])),
    "out": tf.Variable(tf.random_normal([1024,n_classes]))
}

biases = {
    "bc1": tf.Variable(tf.random_normal([32])),
    "bc2": tf.Variable(tf.random_normal([64])),
    "bd1": tf.Variable(tf.random_normal([1024])),
    "out": tf.Variable(tf.random_normal([n_classes]))
}

## Convolution Layers
The conv2d function computes the convolution against weight W, and then adds bias b, before applying a ReLU activation function

In [5]:
def conv2d(x,W,b,strides=1):
    x = tf.nn.conv2d(x,W,strides=[1,strides,strides, 1],padding="SAME")
    x = tf.nn.bias_add(x,b)
    return tf.nn.relu(x)

## Max Pooling Layers
The maxpool2d function applies max pooling to layer x using a filter of size k.

In [6]:
def maxpool2d(x,k=2):
  return tf.nn.max_pool(x,ksize=[1,k,k,1],strides=[1,k,k,1],padding="SAME")

## Model
The transformation of each layer to new dimensions is shown in the comments. For example, the first layer shapes the images from 28x28x1 to 28x28x32 in the convolution step

The next step applies max pooling, turning each sample into 14x14x32

All the layers are applied from conv1 to output, producing 10 class predictions.

In [7]:
def conv_net(x,weights,biases,dropout):
  # Layer 1 - 28*28*1 to 14*14*32
  conv1 = conv2d(x,weights["wc1"],biases["bc1"])
  conv1 = maxpool2d(conv1,k=2)

  # Layer 2 - 14*14*32 to 7*7*64
  conv2 = conv2d(conv1,weights["wc2"],biases["bc2"])
  conv2 = maxpool2d(conv2,k=2)

  # Fully connected layer - 7*7*64 to 1024
  fc1 = tf.reshape(conv2,[-1,weights["wd1"].get_shape().as_list()[0]])
  fc1 = tf.add(tf.matmul(fc1,weights["wd1"]),biases["bd1"])
  fc1 = tf.nn.relu(fc1)
  fc1 = tf.nn.dropout(fc1,dropout)

  # Output Layer - class prediction - 1024 to 10
  out = tf.add(tf.matmul(fc1,weights["out"]),biases["out"])
  return out

## Session
Running phase

### Graph Input

In [8]:
x = tf.placeholder(tf.float32,[None,28,28,1])
y = tf.placeholder(tf.float32,[None,n_classes])
keep_prob = tf.placeholder(tf.float32)

### Model

In [10]:
logits = conv_net(x,weights,biases,keep_prob)

### Define loss and optimizer

In [11]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

### Accuracy

In [12]:
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

### Initializing the variable

In [13]:
init = tf.global_variables_initializer()

# Launching it

In [15]:
with tf.Session() as sess:
  sess.run(init)

  for epoch in range(epochs):
    for batch in range(mnist.train.num_examples//batch_size):
      batch_x,batch_y = mnist.train.next_batch(batch_size)
      sess.run(optimizer,{x:batch_x,y:batch_y,keep_prob:dropout})

      # Calculate batch loss
      loss = sess.run(cost,{x:batch_x,y:batch_y,keep_prob:1.})

      # Calculate batch accuracy
      valid_accuracy = sess.run(accuracy,{x:mnist.validation.images[:test_valid_size],y:mnist.validation.labels[:test_valid_size],keep_prob:1.})

      print('Epoch {:>2}, Batch {:>3} - Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(epoch + 1,batch + 1,loss,valid_accuracy))
  
  # Calculate test accuracy
  test_accuracy = sess.run(accuracy,{x: mnist.test.images[:test_valid_size],y: mnist.test.labels[:test_valid_size],keep_prob: 1.})
  print('Testing Accuracy: {}'.format(test_accuracy))



Epoch  1, Batch   1 - Loss: 36262.5703 Validation Accuracy: 0.210938
Epoch  1, Batch   2 - Loss: 27466.0137 Validation Accuracy: 0.222656
Epoch  1, Batch   3 - Loss: 26202.8008 Validation Accuracy: 0.226562
Epoch  1, Batch   4 - Loss: 24705.4316 Validation Accuracy: 0.242188
Epoch  1, Batch   5 - Loss: 22119.0723 Validation Accuracy: 0.230469
Epoch  1, Batch   6 - Loss: 19068.2812 Validation Accuracy: 0.234375
Epoch  1, Batch   7 - Loss: 21089.8301 Validation Accuracy: 0.253906
Epoch  1, Batch   8 - Loss: 18424.5332 Validation Accuracy: 0.277344
Epoch  1, Batch   9 - Loss: 19259.1875 Validation Accuracy: 0.269531
Epoch  1, Batch  10 - Loss: 17211.1055 Validation Accuracy: 0.269531
Epoch  1, Batch  11 - Loss: 14527.2285 Validation Accuracy: 0.289062
Epoch  1, Batch  12 - Loss: 18496.9785 Validation Accuracy: 0.300781
Epoch  1, Batch  13 - Loss: 15864.3086 Validation Accuracy: 0.292969
Epoch  1, Batch  14 - Loss: 16563.6484 Validation Accuracy: 0.300781
Epoch  1, Batch  15 - Loss: 15596.